# Question Answering with Transformers
### Encoder-Decoder Architecture

Let's revisit Question Answering and see how we can use Attention to create a Transformer instead of an LSTM. The type of Transformer network we will be using is called a Encoder-Decoder Transformer and is the first type of Transformer Network that was first detailed in the paper "Attention is all you need!". In fact the Encoder only and Decoder only networks we've already seen came later! We looked at them first as they are simpler to understand.
<br>
Encoder-Decoder Transformers Combine an Encoder Network with Self-Attention and a Decoder Network with Masked Self-Attention. The Encoder will Encode the input sequence (in our example this is the question!) and the Decoder will perform next-token prediction (in our example this is the answer!). The two are combined with a Cross-Attention Layer. This one-way attention allows the Decoder embeddings to query the Encoder embeddings, giving the Decoder information about what the output sequence (answer) should look like! Cross-Attention is a very common way to "condition" a text-generating Decoder network! <br>
What else could we condition the decoder with...

<img src="../data/llm_architecture_comparison.png" width="600" align="center">
<br>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import io
import re
import math
from tqdm.notebook import trange, tqdm

import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import torch.nn.functional as F
from torch.distributions import Categorical

from torchtext.datasets import YahooAnswers
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import torchtext.transforms as T
from torch.hub import load_state_dict_from_url
from torchtext.data.functional import sentencepiece_tokenizer, load_sp_model

torch.backends.cuda.matmul.allow_tf32 = True

In [ ]:
# Define the learning rate for the optimizer
learning_rate = 1e-4

# Define the number of epochs for training
nepochs = 20

# Define the batch size for mini-batch gradient descent
batch_size = 128

# Define the maximum length of input questions
max_len_q = 32

# Define the maximum length of output answers
max_len_a = 64

# Define the root directory of the dataset
data_set_root = "../../datasets"

## Data processing and Tokenization

In [ ]:
# We'll be using the YahooAnswers Dataset
# Note that for torchtext these datasets are NOT Pytorch dataset classes "YahooAnswers" is a function that
# returns a Pytorch DataPipe!

# Pytorch DataPipes vvv
# https://pytorch.org/data/main/torchdata.datapipes.iter.html

# vvv Good Blog on the difference between DataSet and DataPipe
# https://medium.com/deelvin-machine-learning/comparison-of-pytorch-dataset-and-torchdata-datapipes-486e03068c58

# Depending on the dataset sometimes the dataset doesn't download and gives an error
# and you'll have to download and extract manually 
# "The datasets supported by torchtext are datapipes from the torchdata project, which is still in Beta status"

# Un-comment to triger the DataPipe to download the data vvv
# dataset_train = YahooAnswers(root=data_set_root, split="train")
# data = next(iter(dataset_train))

# Side-Note I've noticed that the WikiText dataset is no longer able to be downloaded :(

In [ ]:
# ## "Train" a Sentence Piece Tokenizer with the train data capping the vocab size to 20000 tokens
# from torchtext.data.functional import generate_sp_model

# with open(os.path.join(data_set_root, "datasets/YahooAnswers/train.csv")) as f:
#     with open(os.path.join(data_set_root, "datasets/YahooAnswers/data.txt"), "w") as f2:
#         for i, line in enumerate(f):
#             text_only = "".join(line.split(",")[1:])
#             filtered = re.sub(r'\\|\\n|;', ' ', text_only.replace('"', ' ').replace('\n', ' ')) # remove newline characters
#             f2.write(filtered.lower() + "\n")


# generate_sp_model(os.path.join(data_set_root, "datasets/YahooAnswers/data.txt"), 
#                   vocab_size=20000, model_prefix='spm_user_ya')

In [ ]:
class YahooQA(Dataset):
    """
    Custom Dataset class for handling the Yahoo Answers dataset.
    """

    def __init__(self, num_datapoints, test_train="train"):
        # Read the dataset from the CSV file
        self.df = pd.read_csv(os.path.join(data_set_root, "datasets/YahooAnswers/" + test_train + ".csv"),
                              names=["Class", "Q_Title", "Q_Content", "A"])
        
        # Fill any missing values with empty strings
        self.df.fillna('', inplace=True)
        
        # Combine question title and content into a single question
        self.df['Q'] = self.df['Q_Title'] + ': ' + self.df['Q_Content']
        
        # Drop the now redundant question title and content columns
        self.df.drop(['Q_Title', 'Q_Content'], axis=1, inplace=True)
        
        # Clean the question and answer text by removing unwanted characters
        self.df['Q'] = self.df['Q'].str.replace(r'\\n|\\|\\r|\\r\\n|\n|"', ' ', regex=True)
        self.df['A'] = self.df['A'].str.replace(r'\\n|\\|\\r|\\r\\n|\n|"', ' ', regex=True)

    def __getitem__(self, index):
        # Retrieve the question and answer text and convert them to lowercase
        question_text = self.df.loc[index]["Q"].lower()
        answer_text = self.df.loc[index]["A"].lower()

        # Return a tuple of the question and answer text
        return question_text, answer_text

    def __len__(self):
        # Return the number of data points in the dataset
        return len(self.df)

In [ ]:
# Create training and testing datasets using the YahooQA class
dataset_train = YahooQA(num_datapoints=data_set_root, test_train="train")
dataset_test = YahooQA(num_datapoints=data_set_root, test_train="test")

# Create data loaders for training and testing datasets
data_loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True)
data_loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True, num_workers=4)

In [ ]:
# Example of using the tokenizer
# Load the SentencePiece model
sp_model = load_sp_model("spm_user_ya.model")

# Create a tokenizer using the loaded model
tokenizer = sentencepiece_tokenizer(sp_model)

# Iterate over tokens generated by the tokenizer
for token in tokenizer(["welcome to this tutorial!"]):
    print(token)

In [ ]:
# Define a function to yield tokens from a file
def yield_tokens(file_path):
    # Open the file in UTF-8 encoding
    with io.open(file_path, encoding='utf-8') as f:
        # Iterate over each line in the file
        for line in f:
            # Yield the token split by tab character
            yield [line.split("\t")[0]]

            
# Build vocabulary from the iterator of tokens
# We will also add "special" tokens that we'll use to signal something to our model
# <pad> is a padding token that is added to the end of a sentence to ensure 
# the length of all sequences in a batch is the same
# <soq> signals the "Start-Of-Question" aka the start of the Question sequence
# <eoq> signals the "End-Of-Question" aka the end of the Question sequence
# <soa> signals the "Start-Of-Answer" aka the start of the Answer sequence
# <eoa> signals the "End-Of-Answer" aka the end of the Answer sequence
# <unk> "unknown" token is used if a token is not contained in the vocab
vocab = build_vocab_from_iterator(
    yield_tokens("spm_user_ya.vocab"),
    # Define special tokens with special_first=True to place them at the beginning of the vocabulary
    specials=['<pad>', '<soq>', '<eoq>', '<soa>', '<eoa>', '<unk>'],
    special_first=True
)

# Set default index for out-of-vocabulary tokens
vocab.set_default_index(vocab['<unk>'])

In [ ]:
q_tranform = T.Sequential(
    # Tokeniz with pre-existing Tokenizer
    T.SentencePieceTokenizer("spm_user_ya.model"),
    ## converts the sentences to indices based on given vocabulary
    T.VocabTransform(vocab=vocab),
    ## Add <sos> at beginning of each sentence. 1 because the index for <sos> in vocabulary is
    # 1 as seen in previous section
    T.AddToken(1, begin=True),
    # Crop the sentance if it is longer than the max length
    T.Truncate(max_seq_len=max_len_q),
    ## Add <eos> at beginning of each sentence. 2 because the index for <eos> in vocabulary is
    # 2 as seen in previous section
    T.AddToken(2, begin=False),
    # Convert the list of lists to a tensor, this will also
    # Pad a sentence with the <pad> token if it is shorter than the max length
    # This ensures all sentences are the same length!
    T.ToTensor(padding_value=0)
)

a_tranform = T.Sequential(
    # Tokeniz with pre-existing Tokenizer
    T.SentencePieceTokenizer("spm_user_ya.model"),
    ## converts the sentences to indices based on given vocabulary
    T.VocabTransform(vocab=vocab),
    ## Add <sos> at beginning of each sentence. 1 because the index for <sos> in vocabulary is
    # 1 as seen in previous section
    T.AddToken(3, begin=True),
    # Crop the sentance if it is longer than the max length
    T.Truncate(max_seq_len=max_len_a),
    ## Add <eos> at beginning of each sentence. 2 because the index for <eos> in vocabulary is
    # 2 as seen in previous section
    T.AddToken(4, begin=False),
    # Convert the list of lists to a tensor, this will also
    # Pad a sentence with the <pad> token if it is shorter than the max length
    # This ensures all sentences are the same length!
    T.ToTensor(padding_value=0)
)

## Create Encoder-Decoder Network

In [ ]:
# sinusoidal positional embeds
class SinusoidalPosEmb(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, x):
        device = x.device
        half_dim = self.dim // 2
        emb = math.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=device) * -emb)
        emb = x[:, None] * emb[None, :]
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        return emb


# Define a module for attention blocks
class AttentionBlock(nn.Module):
    def __init__(self, hidden_size=128, num_heads=4, masking=True):
        super(AttentionBlock, self).__init__()
        self.masking = masking

        # Multi-head attention mechanism
        self.multihead_attn = nn.MultiheadAttention(hidden_size,
                                                    num_heads=num_heads,
                                                    batch_first=True,
                                                    dropout=0.25)

    def forward(self, x_in, kv_in, key_mask=None):
        # Apply causal masking if enabled
        if self.masking:
            bs, l, h = x_in.shape
            mask = torch.triu(torch.ones(l, l, device=x_in.device), 1).bool()
        else:
            mask = None
            
        # Perform multi-head attention operation
        return self.multihead_attn(x_in, kv_in, kv_in, attn_mask=mask, key_padding_mask=key_mask)[0]


# Define a module for a transformer block with self-attention and optional causal masking
class TransformerBlock(nn.Module):
    def __init__(self, hidden_size=128, num_heads=4, is_decoder=False, masking=True):
        super(TransformerBlock, self).__init__()
        self.is_decoder = is_decoder

        # Layer normalization for the input
        self.norm1 = nn.LayerNorm(hidden_size)
        # Self-attention mechanism
        self.attn1 = AttentionBlock(hidden_size=hidden_size, num_heads=num_heads, masking=masking)
        
        # Layer normalization for the output of the first attention layer
        if self.is_decoder:
            self.norm2 = nn.LayerNorm(hidden_size)
            # Self-attention mechanism for the decoder with no masking
            self.attn2 = AttentionBlock(hidden_size=hidden_size, num_heads=num_heads, masking=False)
        
        # Layer normalization for the output before the MLP
        self.norm_mlp = nn.LayerNorm(hidden_size)
        # Multi-layer perceptron (MLP)
        self.mlp = nn.Sequential(nn.Linear(hidden_size, hidden_size * 4),
                                 nn.ELU(),
                                 nn.Linear(hidden_size * 4, hidden_size))
                
    def forward(self, x, input_key_mask=None, cross_key_mask=None, kv_cross=None):
        # Perform self-attention operation
        x = self.attn1(x, x, key_mask=input_key_mask) + x
        x = self.norm1(x)

        # If decoder, perform additional cross-attention layer
        if self.is_decoder:
            x = self.attn2(x, kv_cross, key_mask=cross_key_mask) + x
            x = self.norm2(x)

        # Apply MLP and layer normalization
        x = self.mlp(x) + x
        return self.norm_mlp(x)
    
    
# Define an encoder module for the Transformer architecture
class Encoder(nn.Module):
    def __init__(self, num_emb, hidden_size=128, num_layers=3, num_heads=4):
        super(Encoder, self).__init__()
        
        # Create an embedding layer for tokens
        self.embedding = nn.Embedding(num_emb, hidden_size)
        # Initialize the embedding weights
        self.embedding.weight.data = 0.001 * self.embedding.weight.data

        # Initialize sinusoidal positional embeddings
        self.pos_emb = SinusoidalPosEmb(hidden_size)
        
        # Create multiple transformer blocks as layers
        self.blocks = nn.ModuleList([
            TransformerBlock(hidden_size, num_heads, is_decoder=False, masking=False) for _ in range(num_layers)
        ])
                
    def forward(self, input_seq, padding_mask=None):        
        # Embed the input sequence
        input_embs = self.embedding(input_seq)
        bs, l, h = input_embs.shape

        # Add positional embeddings to the input embeddings
        seq_indx = torch.arange(l, device=input_seq.device)
        pos_emb = self.pos_emb(seq_indx).reshape(1, l, h).expand(bs, l, h)
        embs = input_embs + pos_emb
        
        # Pass the embeddings through each transformer block
        for block in self.blocks:
            embs = block(embs, input_key_mask=padding_mask)
        
        return embs

    
# Define a decoder module for the Transformer architecture
class Decoder(nn.Module):
    def __init__(self, num_emb, hidden_size=128, num_layers=3, num_heads=4):
        super(Decoder, self).__init__()
        
        # Create an embedding layer for tokens
        self.embedding = nn.Embedding(num_emb, hidden_size)
        # Initialize the embedding weights
        self.embedding.weight.data = 0.001 * self.embedding.weight.data

        # Initialize sinusoidal positional embeddings
        self.pos_emb = SinusoidalPosEmb(hidden_size)
        
        # Create multiple transformer blocks as layers
        self.blocks = nn.ModuleList([
            TransformerBlock(hidden_size, num_heads, is_decoder=True, masking=True) for _ in range(num_layers)
        ])
                
        # Define a linear layer for output prediction
        self.fc_out = nn.Linear(hidden_size, num_emb)
        
    def forward(self, input_seq, encoder_output, input_padding_mask=None, encoder_padding_mask=None):        
        # Embed the input sequence
        input_embs = self.embedding(input_seq)
        bs, l, h = input_embs.shape

        # Add positional embeddings to the input embeddings
        seq_indx = torch.arange(l, device=input_seq.device)
        pos_emb = self.pos_emb(seq_indx).reshape(1, l, h).expand(bs, l, h)
        embs = input_embs + pos_emb
        
        # Pass the embeddings through each transformer block
        for block in self.blocks:
            embs = block(embs,
                         input_key_mask=input_padding_mask,
                         cross_key_mask=encoder_padding_mask, 
                         kv_cross=encoder_output)
        
        return self.fc_out(embs)

    
# Define an Encoder-Decoder module for the Transformer architecture
class EncoderDecoder(nn.Module):
    def __init__(self, num_emb, hidden_size=128, num_layers=(3, 3), num_heads=4):
        super(EncoderDecoder, self).__init__()
        
        # Create an encoder and decoder with specified parameters
        self.encoder = Encoder(num_emb=num_emb, hidden_size=hidden_size, 
                               num_layers=num_layers[0], num_heads=num_heads)
        
        self.decoder = Decoder(num_emb=num_emb, hidden_size=hidden_size, 
                               num_layers=num_layers[1], num_heads=num_heads)

    def forward(self, input_seq, target_seq):
        # Generate padding masks for input and target sequences
        input_key_mask = input_seq == 0
        output_key_mask = target_seq == 0

        # Encode the input sequence
        encoded_seq = self.encoder(input_seq=input_seq, 
                                   padding_mask=input_key_mask)
        
        # Decode the target sequence using the encoded sequence
        decoded_seq = self.decoder(input_seq=target_seq, 
                                   encoder_output=encoded_seq, 
                                   input_padding_mask=output_key_mask, 
                                   encoder_padding_mask=input_key_mask)

        return decoded_seq

## Initialise Model and Optimizer

In [ ]:
# Check if GPU is available, set device accordingly
device = torch.device(0 if torch.cuda.is_available() else 'cpu')

# Embedding Size
hidden_size = 512

# Number of Transformer blocks for the (Encoder, Decoder)
num_layers = (4, 4)

# MultiheadAttention Heads
num_heads = 8

# Create model
tf_generator = EncoderDecoder(num_emb=len(vocab), num_layers=num_layers, 
                              hidden_size=hidden_size, num_heads=num_heads).to(device)

# Initialize the optimizer with above parameters
optimizer = optim.Adam(tf_generator.parameters(), lr=learning_rate)

# Scaler for mixed precision training
scaler = torch.cuda.amp.GradScaler()

# Define the loss function
loss_fn = nn.CrossEntropyLoss(reduction="none")

# Initialize the training loss logger
training_loss_logger = []

start_epoch = 0

In [ ]:
## Load Checkpoint
# cp = torch.load("qa_model.pt")
# tf_generator.load_state_dict(cp["model_state_dict"])
# optimizer.load_state_dict(cp["optimizer_state_dict"])
# training_loss_logger = cp["data_logger"]
# start_epoch = cp["epoch"]

In [ ]:
# Let's see how many Parameters our Model has!
num_model_params = 0
for param in tf_generator.parameters():
    num_model_params += param.flatten().shape[0]

print("-This Model Has %d (Approximately %d Million) Parameters!" % (num_model_params, num_model_params//1e6))

## Training

In [ ]:
# Iterate over epochs
for epoch in trange(start_epoch, nepochs, leave=False, desc="Epoch"):
    # Set the model in training mode
    tf_generator.train()
    
    # Iterate over the training data loader
    for q_text, a_text in tqdm(data_loader_train, desc="Training", leave=False):
        # Convert question and answer text to tokens and move to device
        q_text_tokens = q_tranform(list(q_text)).to(device)
        a_text_tokens = a_tranform(list(a_text)).to(device)
        a_input_text = a_text_tokens[:, 0:-1]
        a_output_text = a_text_tokens[:, 1:]

        # Forward pass
        with torch.cuda.amp.autocast():
            pred = tf_generator(q_text_tokens, a_input_text)

            # Generate mask for output text
            output_mask = (a_output_text != 0).float()

            # Compute the loss
            loss = (loss_fn(pred.transpose(1, 2), a_output_text) * output_mask).sum()/output_mask.sum()

        # Backpropagation
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        # Log the training loss
        training_loss_logger.append(loss.item())
    
    # Quick save of the model every epoch
    torch.save({'epoch': epoch + 1,
                'data_logger': training_loss_logger,
                'model_state_dict': tf_generator.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                 }, "qa_model.pt")

## Plot Loss

In [ ]:
_ = plt.figure(figsize=(10, 5))
_ = plt.plot(training_loss_logger[10:])
_ = plt.title("Training Loss")

## Testing

In [ ]:
# Get a batch of question and answer text from the test data loader
q_text, a_text = next(iter(data_loader_test))

In [ ]:
# Choose an index within the batch
index = 0

# Print the question text at the chosen index
print("Question:")
print(q_text[index])

# Print the original answer text at the chosen index
print("\nOriginal Answer:")
print(a_text[index])

In [ ]:
# Set the initial prompt with the question text from the test data loader
# init_prompt = [q_text[index]]
init_prompt = ["why did the chicken cross the road"]

# Tokenize the initial prompt question text
input_q_tokens = q_tranform(init_prompt).to(device)

# Add the Start-Of-Answer token to the prompt to signal the network to start generating the answer
soa_token = 3 * torch.ones(1, 1).long()

# Print the token indices of the question text
print("\nQuestion token indices:")
print(input_q_tokens)

# Look up the token strings corresponding to the token indices
token_strings = vocab.lookup_tokens(input_q_tokens[0].cpu().numpy())

# Print the token strings
print("\nQuestion token strings:")
print(token_strings)

# Concatenate the token strings to form the question string and perform formatting
question = "".join(token_strings).replace("▁", " ").replace("<soq>", "").replace("<eoq>", "")

# Print the formatted question string
print("\nQuestion String:")
print(question)

# Set the temperature for sampling during generation
temp = 0.8

In [ ]:
log_tokens = [soa_token]
tf_generator.eval()

with torch.no_grad():
    # Encode the input question tokens
    encoded_seq = tf_generator.encoder(input_q_tokens.to(device))

    # Generate the answer tokens
    for i in range(100):
        input_tokens = torch.cat(log_tokens, 1)
        
        # Decode the input tokens into the next predicted tokens
        data_pred = tf_generator.decoder(input_tokens.to(device), encoded_seq)
        
        # Sample from the distribution of predicted probabilities
        dist = Categorical(logits=data_pred[:, -1] / temp)
        next_tokens = dist.sample().reshape(1, 1)
        
        # Append the next predicted token to the sequence
        log_tokens.append(next_tokens.cpu())
        
        # Break the loop if the End-Of-Answer token is predicted
        if next_tokens.item() == 4:
            break

In [ ]:
# Convert the list of token indices to a tensor
pred_text = torch.cat(log_tokens, 1)

# Convert the token indices to their corresponding strings using the vocabulary
pred_text_strings = vocab.lookup_tokens(pred_text[0].numpy())

# Join the token strings to form the predicted text
pred_text = "".join(pred_text_strings)

# Print the predicted text
print(pred_text)

In [ ]:
# Origional Question
print(question)

In [ ]:
# Remove special tokens and subword tokenization markers from the predicted text
answer_out = pred_text.replace("▁", " ").replace("<unk>", "").replace("<soa>", "").replace("<eoa>", "")
print(answer_out)